# Preprocessing Data

Notebook ini berisi langkah-langkah preprocessing data untuk dataset `dataset_labeled.csv`.
Dataset ini memiliki format yang tidak konsisten, sehingga diperlukan parsing data secara manual.

In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

# --- 1. Data Loading dan Parsing ---
# Muat data secara manual karena format file yang tidak konsisten.
# File dibaca baris per baris, dan kolom dipisahkan menggunakan regex.

try:
    with open('dataset_labeled.csv', 'r') as file:
        content = file.read()
except FileNotFoundError:
    print("File `dataset_labeled.csv` tidak ditemukan. Pastikan file berada di direktori yang sama.")
    # Exit or handle error gracefully

pattern = re.compile(r'(?:\s*"(.*?)",?|([^,]*),?)([^,]*),([\d.]+)((?:.|\s)*)')
data_rows = []

for line in content.strip().split('\n')[1:]: # Lewati header
    match = pattern.search(line)
    if match:
        text = match.group(1) if match.group(1) is not None else match.group(2)
        label = match.group(3)
        confidence = match.group(4)
        data_rows.append([text, label, confidence])
    else:
        # Fallback for some specific malformed lines
        parts = line.split(',')
        if len(parts) >= 3:
            text = parts[0]
            label = parts[1]
            confidence = parts[2]
            data_rows.append([text, label, confidence])

df = pd.DataFrame(data_rows, columns=['text', 'label', 'auto_label_confidence'])

print("Shape data setelah parsing:")
print(df.shape)
print("\n5 baris pertama:")
print(df.head())

### 2. Pembersihan Data
Langkah-langkah pembersihan data yang dilakukan adalah sebagai berikut:
1.  **Menghapus Duplikat**: Menghapus baris yang memiliki nilai duplikat untuk memastikan keunikan data.
2.  **Konversi Tipe Data**: Mengubah tipe data kolom `auto_label_confidence` menjadi numerik.
3.  **Pembersihan Teks**: Mengubah teks menjadi huruf kecil dan menghapus tanda kutip.

In [ ]:
# --- 2. Data Cleaning ---
# Menghapus baris duplikat
initial_row_count = len(df)
df.drop_duplicates(inplace=True)
print(f"Jumlah baris awal: {initial_row_count}")
print(f"Jumlah baris setelah menghapus duplikat: {len(df)}")

# Mengubah tipe data kolom 'auto_label_confidence' menjadi numerik
df['auto_label_confidence'] = pd.to_numeric(df['auto_label_confidence'], errors='coerce')

# Mengubah teks menjadi huruf kecil dan menghapus tanda kutip
df['text'] = df['text'].str.strip().str.lower().str.replace('^"|"$', '', regex=True)

print("\nInformasi DataFrame setelah pembersihan:")
print(df.info())

### 3. Transformasi Data
Pada langkah ini, kolom `label` akan diubah dari data kategori menjadi numerik menggunakan `LabelEncoder`. Ini diperlukan untuk model machine learning yang hanya dapat memproses data numerik.

In [ ]:
# --- 3. Data Transformation ---
# Menggunakan LabelEncoder untuk mengkodekan kolom 'label'
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

print("Mapping Label ke Nilai Terkode:")
for label, encoded_value in zip(le.classes_, le.transform(le.classes_)):
    print(f"  {label}: {encoded_value}")

print("\nDataFrame setelah transformasi:")
print(df.head())

### 4. Menyimpan Hasil Preprocessing
Data yang sudah bersih dan terstruktur disimpan ke dalam file CSV baru.

In [ ]:
# --- 4. Save Preprocessed Data ---
output_file_path = 'preprocessed_dataset_labeled.csv'
df.to_csv(output_file_path, index=False)

print(f"\nData preprocessing selesai dan disimpan ke dalam file: {output_file_path}")